Transfer learning notebook

In [1]:
"""
RuntimeError: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)`
 or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because
 it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set
 an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or
 CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to
 https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility
"""
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['CUBLAS_WORKSPACE_CONFIG']

':16:8'

Train a model from scratch

In [14]:
import yolov5.train as t

opt = t.parse_opt(known=True)

setattr(opt, "cfg", "start.yaml")
setattr(opt, "weights", "")
setattr(opt, "batch_size", 8)
setattr(opt, "epochs", 100)

t.main(opt)

train: weights=, cfg=../start.yaml, data=yolov5/data/coco128.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
YOLOv5 🚀 2023-1-30 Python-3.8.15 torch-1.13.0+cu117 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4040MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.

github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5


start summary: 214 layers, 7235389 parameters, 7235389 gradients, 16.6 GFLOPs

AMP: checks passed ✅
optimizer: SGD(lr=0.01) with parameter groups 57 weight(decay=0.0), 60 weight(decay=0.0005), 60 bias
train: Scanning /home/erik/PycharmProjects/dynamic-yolo/app/prototype/v3/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
val: Scanning /home/erik/PycharmProjects/dynamic-yolo/app/prototype/v3/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]

AutoAnchor: 4.27 anchors/target, 0.994 Best Possible Recall (BPR). Current anchors are a good fit to dataset ✅
Plotting labels to yolov5/runs/train/exp3/labels.jpg... 
Plotting labels to yolov5/runs/train/exp4/labels.jpg... 


Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to yolov5/runs/train/exp4
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       0/99      1.78G     0.1114    0.06513     0.1056        123        640:  25%|██▌       | 4/16 [00:12<00:37,  3.14s/it]


RuntimeError: DataLoader worker (pid(s) 13994, 13995, 13996, 13997) exited unexpectedly

In [16]:
# test model
import yolov5.detect as d

opt2 = d.parse_opt(known=True)
setattr(opt2, "weights", "best300.pt")
# setattr(opt2, "weights", "yolov5s.pt")
setattr(opt2, "conf_thres", 0.30)
setattr(opt2, "save_txt", True)
setattr(opt2, "save_conf", True)
setattr(opt2, "source", "datasets/coco128/images/train2017")
d.main(opt2)

detect: weights=yolov5/yolov5s.pt, source=yolov5/data/images, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 2023-1-30 Python-3.8.15 torch-1.13.0+cu117 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4040MiB)

Fusing layers... 
start summary: 157 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
image 1/128 /home/erik/PycharmProjects/dynamic-yolo/app/prototype/v3/datasets/coco128/images/train2017/000000000009.jpg: 480x640 3 bowls, 2 carrots, 21.4ms
image 2/128 /home/erik/PycharmProjects/dynamic-yolo/app/prototype/v3/datasets/coco128/images/train2017/000000000025.jpg: 448x640 1 elephant, 1 giraffe, 18.0ms
image 3/128 /home/erik/Pyc

Train new model with different data, but use previous weights

saved results to exp15

In [1]:
# train with different dataset

import yolov5.train as t

opt = t.parse_opt(known=True)

setattr(opt, "cfg", "")
setattr(opt, "weights", "best300.pt")
setattr(opt, "data", "VOC.yaml")
setattr(opt, "batch_size", 8)
setattr(opt, "epochs", 3)
setattr(opt, "name", "yolo_test")

t.main(opt)

train: weights=best300.pt, cfg=, data=VOC.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=3, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=yolo_test, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest


github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5


YOLOv5 🚀 2023-1-30 Python-3.8.16 torch-1.13.0+cu117 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4040MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=1.0, mixup=0.0, copy_paste=0.0
ClearML: run 'pip install clearml' to automatically track, visualize and remotely train YOLOv5 🚀 in ClearML
Comet: run 'pip install comet_ml' to automatically track and visualize YOLOv5 🚀 runs in Comet
TensorBoard: Start with 'tensorboard --logdir yolov5/runs/train', view at http://localhost:6006/
Overriding model.yaml nc=80 with nc=20

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv             

Test trained model with new and old data

In [3]:
import yolov5.detect as d

opt2 = d.parse_opt(known=True)
setattr(opt2, "weights", "300_100.pt")
# setattr(opt2, "weights", "yolov5s.pt")
setattr(opt2, "conf_thres", 0.30)
setattr(opt2, "save_txt", True)
setattr(opt2, "save_conf", True)
setattr(opt2, "source", "data/images")

d.main(opt2)

detect: weights=yolov5/yolov5s.pt, source=yolov5/data/images, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 2023-1-30 Python-3.8.16 torch-1.13.0+cu117 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4040MiB)

Fusing layers... 
Model summary: 157 layers, 7064065 parameters, 0 gradients, 15.9 GFLOPs
image 1/999 /home/erik/PycharmProjects/dynamic-yolo/app/prototype/v3/data/images/000015.jpg: 480x640 1 bicycle, 32.7ms
image 2/999 /home/erik/PycharmProjects/dynamic-yolo/app/prototype/v3/data/images/000029.jpg: 448x640 1 dog, 16.2ms
image 3/999 /home/erik/PycharmProjects/dynamic-yolo/app/prototype/v3/data/images/000038.jpg: 480x640 2 bicy

In [4]:
opt2 = d.parse_opt(known=True)
setattr(opt2, "weights", "300_100.pt")
# setattr(opt2, "weights", "yolov5s.pt")
setattr(opt2, "conf_thres", 0.30)
setattr(opt2, "save_txt", True)
setattr(opt2, "save_conf", True)
setattr(opt2, "source", "datasets/coco128/images/train2017")

d.main(opt2)

detect: weights=yolov5/yolov5s.pt, source=yolov5/data/images, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 2023-1-30 Python-3.8.16 torch-1.13.0+cu117 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4040MiB)

Fusing layers... 
Model summary: 157 layers, 7064065 parameters, 0 gradients, 15.9 GFLOPs
image 1/128 /home/erik/PycharmProjects/dynamic-yolo/app/prototype/v3/datasets/coco128/images/train2017/000000000009.jpg: 480x640 (no detections), 12.8ms
image 2/128 /home/erik/PycharmProjects/dynamic-yolo/app/prototype/v3/datasets/coco128/images/train2017/000000000025.jpg: 448x640 (no detections), 11.2ms
image 3/128 /home/erik/PycharmProje

In [11]:
import torch

model = torch.hub.load('.', 'custom', path="300_100.pt", source='local')


print(model.names)

YOLOv5 🚀 2023-1-30 Python-3.8.16 torch-1.13.0+cu117 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4040MiB)

Fusing layers... 
Model summary: 157 layers, 7064065 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


{0: 'aeroplane', 1: 'bicycle', 2: 'bird', 3: 'boat', 4: 'bottle', 5: 'bus', 6: 'car', 7: 'cat', 8: 'chair', 9: 'cow', 10: 'diningtable', 11: 'dog', 12: 'horse', 13: 'motorbike', 14: 'person', 15: 'pottedplant', 16: 'sheep', 17: 'sofa', 18: 'train', 19: 'tvmonitor'}
